In [1]:
import numpy as np
import time
import copy
#Leonardo Vanzin
#Mateus Karvat Camara
class Tree:
    # classe Tree cria um nó quando é chamada
    def __init__(self, data):
        self.children = []
        self.parents = []
        self.data = data

    def __str__(self):
    # quando print é chamada para um objeto do tipo Tree, ele exibe o dado do nó 
    # atual e dos filhos apenas desse nó
        children_data = []
        for child in self.children:
            child_data = str(child.data)
            children_data.append(child_data)
        a = "Data: "+str(self.data)+"\n"+"Children: "+str(children_data)
        return a

def verifica_validade(x,y):
    #verifica se estao na mesma linha
    if x//8==y//8:
        return False
    #verifica se estao na mesma coluna
    elif x%8==y%8:
        return False
    #verifica se estao na mesma diagonal principal (\)
    elif abs(x-y)%9==0 and max(x,y)%8>min(x,y)%8:
        return False
    #verifica se estao na mesma diagonal secundaria (/)
    elif abs(x-y)%7==0 and max(x,y)%8<min(x,y)%8:
        return False
    #se todas as verificacoes passaram a posicao e valida
    else:
        return True

def cria_arvore(node_pai, indice_pai, nivel):
    #a arvore é criada no nivel 1 e deve ter no máximo 8 níves. logo, se nivel==9, a criação da árvore deve cessar
    if nivel==9:
        return None
    #verifica-se se o pai pode ter filhos. caso a posição de um nó seja inválida, tal nó é armazenado com um *, o que torna seu tipo de dado
    #uma string.
    if type(node_pai.data) != str:
        #cria-se filhos para o nó pai atual com índices maiores que o do nó pai. Logo, a árvore é criada apenas com numeração ascendente
        #Isto é: 0->10->25->...
        #isso elimina a verificação repetida em combinações já testadas, como 0->25->10->...
        for indice_atual in range(indice_pai+1,64):
            #cria-se um nó vazio
            node_atual = Tree(None)
            #copia-se os nós do pai para o nó recém-criado
            node_atual.parents=copy.deepcopy(node_pai.parents)
            #adiciona-se o pai na lista de pais do nó atual
            node_atual.parents.append(indice_pai)
            valido=True
            #verifica-se se a posição do nó atual é válida em relação a cada um dos nós pai que o precederam na árvore
            for parent in node_atual.parents:
                if not verifica_validade(parent,indice_atual):
                    valido=False
                    break
            #caso o nó seja válido, adiciona-se o índice atual a ele
            if valido:
                node_atual.data = indice_atual
            else:
                node_atual.data=str(indice_atual)+"*"
            #após a criação do nó, ele é adicionado a lista de filhos de seu pai
            node_pai.children.append(node_atual)

            #incrementa-se o contador de nível da árvore
            nivel += 1
            #chama-se a função de forma recursiva para criar o próximo nível da árvore a partir do nó atual
            cria_arvore(node_atual, indice_atual, nivel)

def inicia_arvore():
    #visto que o primeiro nível da árvore não segue o mesmo padrão que os demais, ele é criado fora da função recursiva
    raiz = Tree(None)
    for i in range(64):
        node = Tree(i)

        raiz.children.append(node)

        cria_arvore(node, i, 1)
    
    return raiz

def exibir_respostas_validas(respostas_validas): #função para organizar as listas para imprimir bonitinho
    for resposta in respostas_validas:#percorre as lista que estao em respostas_validas
        resposta_para_printar = [] #cria uma lista para cada lista de caminhos validos
        for nozinho in resposta:#percorre  o conteudo das lista que estao dentro da lista respostas_validas
            resposta_para_printar.append(nozinho.data) #coloca o valor de cada no na lista, isso porque o conteudo das listas eram os endereços dos nos
        print(resposta_para_printar) #imprime os caminhos validos

def busca_em_profundidade_preliminar(no_inicial):
    caminhos=[0] #coloca o valor da posicao zero de respostas invalidas como 0 incialmente, 
                 #foi utilizado uma lista pois ao passar uma variavel como parametro para uma funcao é somente passado o valor dela e nao a referencia
    pilha = [] #cria a pilha que vai ser utilizada na busca em profundidade
    respostas_validas = [] #cria lista que sera armazenado os caminhos validos
    busca_em_profundidade_recursivo(no_inicial, pilha, respostas_validas, caminhos)
    print("Número de respostas válidas:",len(respostas_validas)) #imprime o numero de listas que que estao dentro de respostas_validas 
    print("Número de respostas inválidas:", caminhos[0]) #imprime a primeira posicao da lista, que contem o numero de caminhos invalidos
    print("Caminhos válidos:")
    exibir_respostas_validas(respostas_validas) #chama funcao para imprimir os caminhos validos

def busca_em_profundidade_recursivo(no_referencia, pilha, respostas_validas, caminhos):
    pilha.append(no_referencia) #passa o no inicial
    for no_filho in no_referencia.children: #percorre a arvore, indo de filho em filho
        busca_em_profundidade_recursivo(no_filho, pilha, respostas_validas, caminhos)
    if (len(pilha)>9) : #seleciona os caminhos que sao maior que 9, porque sao os caminhos que possuem os 8 niveis
        if pilha[1:-1] not in respostas_validas: #verifica se ja existe um dentro da lista, para evitar respostas repetidas
            respostas_validas.append(pilha[1:-1]) #coloca na lista de respostas validas tirando o no raiz e o ultimo filho, o ultimo filho é tirado pois o pai dele ja é o 8 nivel
    elif (len(pilha)==9 and pilha[-1].data==63):#como o 63 é o ultimo no possivel ele nao tem filho, mas tem casos em que o 63 é valido para a resposta, 
                                                #assim verificamos se no 8 nivel tem o 63 e com isso colocamos ele na lista
        if pilha[1:] not in respostas_validas: #neste caso como o 63 é o ultimo nivel e ele nao possui filho, tiramos somente o no raiz
            respostas_validas.append(pilha[1:])
    else:
        caminhos[0]+=1 #contabiliza quantos caminhos invalidos foram encontrados na primeira posicao da lista
    pilha.pop(pilha.index(no_referencia)) #seguindo o algoritmo da busca em profundidade elimina-se o no da pilha

In [2]:
raiz = inicia_arvore()

In [3]:
busca_em_profundidade_preliminar(raiz)

Número de respostas válidas: 92
Número de respostas inválidas: 924223
Caminhos válidos:
[0, 12, 23, 29, 34, 46, 49, 59]
[0, 13, 23, 26, 38, 43, 49, 60]
[0, 14, 19, 29, 39, 41, 52, 58]
[0, 14, 20, 31, 33, 43, 53, 58]
[1, 11, 21, 31, 34, 40, 54, 60]
[1, 12, 22, 24, 34, 47, 53, 59]
[1, 12, 22, 27, 32, 47, 53, 58]
[1, 13, 16, 30, 35, 47, 50, 60]
[1, 13, 23, 26, 32, 43, 54, 60]
[1, 14, 18, 29, 39, 44, 48, 59]
[1, 14, 20, 31, 32, 43, 53, 58]
[1, 15, 21, 24, 34, 44, 54, 59]
[2, 8, 22, 28, 39, 41, 51, 61]
[2, 12, 17, 31, 32, 46, 51, 61]
[2, 12, 17, 31, 37, 43, 54, 56]
[2, 12, 22, 24, 35, 41, 55, 61]
[2, 12, 23, 27, 32, 46, 49, 61]
[2, 13, 17, 28, 39, 40, 54, 59]
[2, 13, 17, 30, 32, 43, 55, 60]
[2, 13, 17, 30, 36, 40, 55, 59]
[2, 13, 19, 24, 39, 44, 54, 57]
[2, 13, 19, 25, 39, 44, 54, 56]
[2, 13, 23, 24, 35, 46, 52, 57]
[2, 13, 23, 24, 36, 46, 49, 59]
[2, 13, 23, 25, 35, 40, 54, 60]
[2, 14, 17, 31, 36, 40, 51, 61]
[2, 14, 17, 31, 37, 43, 48, 60]
[2, 15, 19, 30, 32, 45, 49, 60]
[3, 8, 20, 31, 33

In [4]:
def Randomiza():
    #gera numero aleatorio entre 0 e 1
    return np.random.rand()

def RandomS0():
    rng = np.random.default_rng()
    #gera uma lista de 8 inteiros distintos aleatoriamente
    x = rng.choice(64, size=8, replace=False)
    x_list = list(x)
    #ordena a lista de modo a torná-la estritamente crescente
    x_list.sort()

    return x_list

def func_obj(Si,tree):
    cost = 0
    x = Si[0]   #x é o elemento de interesse da lista Si
    node = tree.children[x]  #acessa nó x no primeiro nivel da árvore
    for nivel in range(1,8):
        x = Si[nivel]   #atualiza x

        #cria lista com os dados dos filhos do nó de interesse
        children_data = []
        for child in node.children:
            children_data.append(child.data)

        #se x está entre os nós filhos do atual nó, passa-se o atual nó para o nó
        #cujo dado é x
        if x in children_data:
            indice_x = children_data.index(x)
            node = node.children[indice_x]
        #do contrário, o nó que contém x é um nó folha e calcula-se o custo de tal nó
        #o custo é calculado pelo número de filhos que o nó em questão não tem
        #ou seja, se ele está no 7º nível, ele não tem um filho no 8º nível nem no 9º e seu custo é 20
        else:
            cost = (9-nivel)*10
            break
    return cost

def Perturba(S):
    #sorteia o índice do elemento a ser perturbado
    indice = np.random.randint(8)

    #variavel booleana pra garantir a checagem correta dos IFs
    check = False

    while S[indice]-indice>=56:
        indice = np.random.randint(8)

    #se não é o último elemento
    if indice<7:
        #se o valor é 0, obriga a somar 1, exceto se o próximo valor é 1, 
        #aí não faz nada para evitar criar valores repetidos
        if S[indice]==0:
            check=True
            if S[indice+1]!=1:
                S[indice]+=1
        #se o elemento posterior é o número suscessor ao número atual, 
        #obriga a subtrair em 1 o número atual, visto que ele não é 0
        elif S[indice+1]==S[indice]+1:
            if indice>0:
                if S[indice-1]!=S[indice]-1:
                    S[indice]-=1
            check=True
        else:
            if indice>0:
                if S[indice-1]!=S[indice]-1:
                    S[indice]-=1
            check=True
    
    #se não é o primeiro elemento
    else:
        #se o elemento anterior é o número antecessor ao número atual, 
        # obriga a somar 1 no número atual, desde que ele não seja 63
        if S[indice-1]==S[indice]-1:
            check=True
            if S[indice]!=63:
                S[indice]+=1
        #se for 63 (mas o anterior não era 62), subtrai 1 dele
        elif S[indice]==63:
            S[indice]-=1
            check=True
    
    #se não for nenhum dos casos especiais acima, que obrigaria a somar ou subtrair,
    #escolhe aleatoriamente entre subtração e soma de 1
    if not check:
        #escolhe um número aleatório entre -1 e 1, então aplica função sinal nele,
        #obtendo -1 ou 1 aleatoriamente. então adiciona o resultado ao elemento 
        #perturbado
        perturbacao = int(np.sign(np.random.rand()*2-1))
        S[indice]+= perturbacao
    
    rng = np.random.default_rng()
    tamanho = 8-(indice+1)  #número de elementos a serem substituídos
    intervalo = 63-S[indice]    #intervalo de valores possível
    
    #cria uma lista de valores a serem substituídos após o valor perturbado
    try:
        x = rng.choice(intervalo, size=tamanho, replace=False)
    except:
        print(f"tamanho={tamanho}, intervalo={intervalo}")
        print(S)
        print(f"indice={indice}, S[indice]={S[indice]}")
    x += int(S[indice]+1)
    x_list = list(x)
    x_list.sort()

    #substitui os valores de S pelos valores de x_list
    for i in range(len(x_list)):
        S[indice+1+i]=x_list[i]

    return S

In [5]:
def tempera_simulada(M, P, L, alpha, T0, tree):
    #M= maximo de iterações do laço externo
    #P= maximo de iterações do laço interno
    #L= limite do número de sucessos
    #alpha= fator de redução da temperatura
    #T0= temperatura inicial
    #tree= raiz da árvore a ser buscada
    T=T0
    j=1
    S=RandomS0()

    while j<=M:
        i=1
        nSucesso=0
        while nSucesso<L and i<=P:
            Si=Perturba(S)
            delta_Fi = func_obj(Si,tree) - func_obj(S,tree)
            exp = np.exp(-delta_Fi/T)
            if delta_Fi<=0 or exp>Randomiza():
                S=Si
                nSucesso += 1
            i+=1
        T*=alpha

        j+=1
        if nSucesso==0:
            break
    
    #print("-"*10)
    #print(f"M={M},P={P},L={L},alpha={alpha},T={T0}")
    #print(f"i={i},j={j},T={round(T,4)},cost={func_obj(S,tree)}\n{S}")
    return func_obj(S,tree), S

In [30]:
tempera_simulada(100, 200, 8, 0.9, 100, raiz)

i=9,j=101,T=0.0027,cost=80
[17, 22, 23, 29, 39, 43, 54, 55]


In [6]:
#M,P,L,alpha,T
M=[50,100,200,500,750,1000]
P=[5,10,20,40,80,160,300]
L=[1,2,4,8,16,32]
alpha=[0.999,0.99,0.98,0.95,0.9,0.8,0.67]
T=[1,5,10,20,40,70,100,150,200]

#15876 possibilidades

In [7]:
import itertools
i=0
for m,p,l,a,t in list(itertools.product(M[],P,L,alpha,T)):
    i+=1
    custo=tempera_simulada(m,p,l,a,t,raiz)
    if custo<50:
        print(custo,m,p,l,a,t)
    if i%500==0:
        print(f"Iteração {i}")

Iteração 500
Iteração 1000
40 50 40 2 0.8 100
40 50 40 4 0.999 1
Iteração 1500
Iteração 2000
40 50 160 4 0.999 5
30 50 160 32 0.98 200
Iteração 2500
Iteração 3000
40 100 20 1 0.8 200
Iteração 3500
40 100 20 4 0.9 20
40 100 40 1 0.999 100
Iteração 4000
Iteração 4500
40 100 160 8 0.99 1
Iteração 5000
Iteração 5500
40 200 5 32 0.9 10
Iteração 6000
Iteração 6500
Iteração 7000
40 200 160 8 0.8 5
Iteração 7500
Iteração 8000
Iteração 8500
Iteração 9000
30 500 20 32 0.99 100
40 500 40 16 0.8 1
Iteração 9500
Iteração 10000
<ipython-input-4-69cff42ed4f8>:29: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if x in children_data:


In [9]:
T=time.time()
for i in range(100):
    #cria-se listas vazias para armazenar o custo e estado obtido após cada execução do algoritmo
    custos=[]
    estados=[]

    #o algoritmo é chamado com os parâmetros que apresentaram melhor resultado na busca por parâmetros (descrita no relatório)
    custo, S = tempera_simulada(50,160,32,0.98,200, raiz)

    #armazena-se o custo e estado da presente execução do algoritmo nas listas
    custos.append(custo)
    estados.append(S)

T2=time.time()
print(min(custos),estados[custos.index(min(custos))], round(T2-T,5))

70 [20, 31, 34, 35, 36, 38, 52, 55] 139.06337


In [17]:
x=[]
count40=0
count30=0
t0=time.time()
for i in range(100000):
    cost=func_obj(RandomS0(),raiz)
    x.append(cost)
    if cost==40:
        count40+=1
    elif cost==30:
        count30+=1
    elif cost<30:
        print(cost)
t1=time.time()
print(min(x))
print(t1-t0)
print(count30,count40)

20
20
40.63349199295044
9 172


In [22]:
S0 = RandomS0()
print(S0)
for i in range(10):
    S0=Perturba(S0)
    print(S0)

[3, 14, 27, 38, 46, 47, 59, 61]
[3, 14, 27, 38, 45, 49, 55, 58]
[3, 14, 27, 38, 44, 48, 58, 60]
[3, 14, 27, 37, 41, 44, 48, 61]
[3, 13, 16, 17, 33, 35, 37, 49]
[3, 13, 16, 17, 33, 34, 37, 53]
[3, 13, 16, 17, 18, 29, 52, 63]
[3, 13, 16, 17, 18, 28, 38, 48]
[3, 13, 16, 17, 18, 28, 38, 49]
[3, 13, 16, 17, 18, 52, 53, 62]
[3, 12, 14, 20, 30, 36, 50, 56]


In [1]:
import numpy as np 
import time

In [6]:
lista = []
for i in range(10):
    lista.append([i,i*2])
print(lista)

[[0, 0], [1, 2], [2, 4], [3, 6], [4, 8], [5, 10], [6, 12], [7, 14], [8, 16], [9, 18]]


In [3]:
def verifica_validade(x,y):
    #verifica se estao na mesma linha
    if x//8==y//8:
        return False
    #verifica se estao na mesma coluna
    elif x%8==y%8:
        return False
    #verifica se estao na mesma diagonal principal (\)
    elif abs(x-y)%9==0 and max(x,y)%8>min(x,y)%8:
        return False
    #verifica se estao na mesma diagonal secundaria (/)
    elif abs(x-y)%7==0 and max(x,y)%8<min(x,y)%8:
        return False
    #se todas as verificacoes passaram a posicao e valida
    else:
        return True



In [4]:
lista=[[0, 6], [0, 32], [0, 36], [27, 20], [13, 4], [29, 50], [57, 30], [23, 41], [41, 55], [14, 16], [2, 59], [0, 61]]
for listinha in lista:
    print(verifica_validade(listinha[0], listinha[1]))

False
False
False
False
False
False
True
True
True
True
True
True


In [9]:
s="pikachu"
if type(s)==str:
    print("aspas funcionou")

aspas funcionou


In [8]:
print(type(s))

<class 'str'>


In [10]:
for element in lista:
    print(element[1])

0
2
4
6
8
10
12
14
16
18


In [5]:
for child in lista:
    print(child)

1
3
4
5
6
7
8
9
10


In [2]:
print("helloword")

helloword


In [11]:
x = [1, 3, 4, 5, 6, 7, 8, 9, 10]
x.append(11)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [22]:
print(f"Oi Leo bjs{x[0]} {x[1]} {x[2]}")

Oi Leo bjs1 3 4


In [31]:
s=input()

In [35]:
print(s)

12 13 14 15


In [37]:
v = s.split(" ")
print(v)
print(type(v))

['12', '13', '14', '15']
<class 'list'>


In [41]:
u=[]
print("Dentro do for")
for element in v:
    print(element)
    u.append(int(element))
print("-"*10)
print(element)
print(u)
print(type(u))
print(u[0])
print(type(u[0]))

Dentro do for
12
13
14
15
----------
15
[12, 13, 14, 15]
<class 'list'>
12
<class 'int'>


In [30]:
s_int = float(s)
print(type(s_int))
print(s_int)

<class 'float'>
12.0


In [45]:
print(range(50,100))
print(type(range(50,100)))

range(50, 100)
<class 'range'>


In [48]:
for i in range(10):
    if i%2==0:
        print(i)
    elif i%3==0:
        print(i*10)
    else:
        print(i*100)

0
100
2
30
4
500
6
700
8
90


In [59]:
def soma(a,b):
    return a+b

In [60]:
def mul(a,b):
    return a*b

In [61]:
def div(a,b):
    return a/b

In [68]:
dict = {
    "A": mul,
    "B": soma,
    "C": div
    }

In [71]:
print(dict["C"](2,2))

1.0


In [72]:
P=print

In [73]:
P("Hello World!")

Hello World!


In [78]:
x,y,z=0,1,2

In [79]:
x,y=y,x

In [80]:
print(x)
print(y)
print(z)

1
0
2


In [82]:
x=0
while(x<10):
    print(x)
    x+=1

0
1
2
3
4
5
6
7
8
9


In [94]:
a = True
b = False
c = 124134314123
d = 0

In [98]:
if (not c or a) and (not d):
    print("A")

A


In [90]:
print(type(c))

<class 'int'>


In [99]:
np.log2(4)

2.0

In [133]:
A = np.array([[0,1,2,3],
              [3,4,5,6],
              [8,9,0,1]])

In [135]:
print(A)

[[0 1 2 3]
 [3 4 5 6]
 [8 9 0 1]]


In [136]:
print(np.shape(A))

(3, 4)


In [132]:
np.ones((2,3))

array([[1., 1., 1.],
       [1., 1., 1.]])

In [130]:
print(A)

[[0 1 2 3]]


In [109]:
x = "abcdefghi"

In [126]:
a = x.index("d")
x = x[:a]+"BLA"+x[a:]

In [127]:
print(x)

jklabcBLAdBLAefghijkljkl
